<a href="https://colab.research.google.com/github/cras-lab/ForMe/blob/main/Consistency_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install -q XlsxWriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 2.7 MB/s eta 0:00:00


In [45]:
from google.colab import files
from google.colab import drive
import pandas as pd
import numpy as np
import math
import shutil
import os

USE_FIXED_FILE = False
SHEET_NAME = [ '전체', 'AI 전문가', '일반 전문가' ]
idx_label = [ '전체L', '전체W', 'AI_L', 'AI_W', '일반L', '일반W' ]

if USE_FIXED_FILE:
  local_file_path = "D:\\Research\\answers.xlsx"
  local_file_path = "C:/nsispromotion_log.txt"
  src_path = "/content/answers.xlsx"
  shutil.copy( local_file_path, src_path )
else:
  uploaded = files.upload()
  filename = list(uploaded.keys())[0]
  src_path = "/content/" + filename

NUM_CLASS = 5
NUM_Q = 4
NUM_COMB = math.comb( NUM_Q, 2)

def GetMatrix( col ):
  matrix = [ [1, col[0], col[1], col[2] ],
             [1/col[0], 1, col[3], col[4]],
             [1/col[1], 1/col[3], 1, col[5]],
             [ 1/col[2], 1/col[4], 1/col[5], 1]
           ]
  return matrix

def FindEigenValue( col ):
  mat = GetMatrix(col)
  eigenvalues, eigenvectors = np.linalg.eig(mat)
  principal_eigenvalue = max(abs(eigenvalues))

  return principal_eigenvalue

def FindEigenValues( col ):
  eigens = np.zeros(NUM_CLASS)

  for i in range( 0, NUM_CLASS):
    col1 = col[i*NUM_COMB:(i+1)*NUM_COMB]
    eigens[i] = FindEigenValue(col1)

  return eigens

def GetWeight(col):
  harmonic = np.zeros(NUM_Q*NUM_CLASS)
  weight = np.zeros(NUM_Q*NUM_CLASS)

  for i in range( 0, NUM_CLASS):
    col1 = col[i*NUM_COMB:(i+1)*NUM_COMB]
    mat = GetMatrix(col1)

    for j in range(0, NUM_Q):
      harmonic[i*NUM_Q+j] = (mat[j][0]*mat[j][1]*mat[j][2]*mat[j][3])**(1/NUM_Q)

    total = sum( harmonic[i*NUM_Q:i*NUM_Q+4])

    weight[i*NUM_Q:i*NUM_Q+4 ] = [ x/total for x in harmonic[i*NUM_Q:i*NUM_Q+4]]

  return weight

drive.mount( '/content/drive')
dest_path = '/content/drive/MyDrive/result.xls'

wgt_sorted_total = []

for num in range(0, len(SHEET_NAME)):
  df = pd.read_excel(src_path,sheet_name=SHEET_NAME[num] )
  num_persons = len(df.columns)

  e_values = np.zeros( (num_persons-1, NUM_CLASS) )

  CI = np.zeros( (num_persons-1, NUM_CLASS) )

  Wgt = np.zeros( (num_persons-1, NUM_CLASS*NUM_Q ) )

  for i in range(0, num_persons-1):
      col = df[ df.columns[i+1]].values
      e_values[i] = FindEigenValues( col )
      CI[i] = [ (x-4)/3/0.9 for x in e_values[i] ]
      Wgt[i] = GetWeight(col)

  ev_df = pd.DataFrame( e_values )
  ev_df.columns = [ 'SERM', 'Subject', 'Enviroment', 'Resource', 'Mechanism']
  ev_df.set_index( df.columns[1:],  inplace=True)

  CI_df = pd.DataFrame( CI )
  CI_df.columns = [ 'SERM', 'Subject', 'Enviroment', 'Resource', 'Mechanism']
  CI_df.set_index( df.columns[1:],  inplace=True)

  Wgt_df = pd.DataFrame( Wgt )
  Wgt_df.set_index( df.columns[1:],  inplace=True)
  Wgt_df.columns = [ 'S', 'E', 'R', 'M', 'S1', 'S2', 'S3', 'S4', 'E1', 'E2', 'E3', 'E4', 'R1', 'R2', 'R3', 'R4', 'M1', 'M2', 'M3', 'M4']

  Wgt_sorted = pd.DataFrame()   # 가중치에 따라 정렬

  for idx in range(0, len(Wgt_df.columns), 4):
    group = Wgt_df.iloc[:, idx:idx+4]
    last_row = group.iloc[-1]
    sorted_group = group.sort_values(by=last_row.name, axis=1, ascending=False)
    Wgt_sorted = pd.concat( [Wgt_sorted, sorted_group], axis=1)

  wgt_sorted_total.append( Wgt_sorted.columns.values )
  wgt_sorted_total.append( Wgt_sorted.iloc[-1, :])
  tmp_path = "/content/temp.xls"

  if num == 0:  # 처음 호출 , 파일 생성
    with pd.ExcelWriter(tmp_path, engine='xlsxwriter') as writer:
    #    ev_df.to_excel(writer, sheet_name='Eigen Values', index=True)
      CI_df.to_excel(writer, sheet_name='CI_' + SHEET_NAME[num] , index=True)
      Wgt_sorted.to_excel(writer, sheet_name='Weight_' + SHEET_NAME[num], index=True)
  else:   # 파일 이미 존재
    with pd.ExcelWriter( tmp_path, engine='openpyxl', mode='a') as writer:
  #    ev_df.to_excel(writer, sheet_name='Eigen Values', index=True)
      CI_df.to_excel(writer, sheet_name='CI_' + SHEET_NAME[num] , index=True)
      Wgt_sorted.to_excel(writer, sheet_name='Weight_' + SHEET_NAME[num], index=True  )

wgt_sorted_total_df = pd.DataFrame(wgt_sorted_total)
wgt_sorted_total_df.set_index( pd.Index(idx_label), inplace=True )
with pd.ExcelWriter( tmp_path, engine='openpyxl', mode='a') as writer:
  wgt_sorted_total_df.to_excel(writer, sheet_name='Total', index=True)

shutil.copy( tmp_path , dest_path )


Saving answers.xlsx to answers (37).xlsx
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/MyDrive/result.xls'